In [22]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import ipynb.fs.full.project_functions as prf

In [23]:
engine = create_engine('postgresql:///opportunity_youth') # since database is owned by me, no other credentials are needed

In [24]:
hs_diploma = """
SELECT pwgtp, agep, schl
FROM pums_2017
WHERE (puma BETWEEN '11610' AND '11615')
    AND (agep BETWEEN 16 AND 24)
    AND (esr  = '3' OR esr = '6')
    AND (schl = '16' OR schl = '17')
    AND sch = '1'
;"""


In [25]:
no_degree = """
SELECT pwgtp, agep, schl
FROM pums_2017
WHERE (puma BETWEEN '11610' AND '11615')
    AND (agep BETWEEN 16 AND 24)
    AND (esr  = '3' OR esr = '6')
    AND (schl <= '15')
    AND sch = '1'
;"""


In [26]:
some_college = """
SELECT pwgtp, agep, schl
FROM pums_2017
WHERE (puma BETWEEN '11610' AND '11615')
    AND (agep BETWEEN 16 AND 24)
    AND (esr  = '3' OR esr = '6')
    AND (schl = '18' OR schl = '19')
    AND sch = '1'
;
"""

In [27]:
college_deg = """
SELECT pwgtp, agep, schl
FROM pums_2017
WHERE (puma BETWEEN '11610' AND '11615')
    AND (agep BETWEEN 16 AND 24)
    AND (esr  = '3' OR esr = '6')
    AND (schl BETWEEN '20' AND '24')
    AND sch = '1'
;
    """

In [28]:
total_oy = """
SELECT pwgtp, agep, schl
FROM pums_2017
WHERE (puma BETWEEN '11610' AND '11615')
    AND (agep BETWEEN 16 AND 24)
    AND (esr  = '3' OR esr = '6')
    AND sch = '1'
;
    """

In [29]:
df_total_oy = pd.read_sql(sql = total_oy, con = engine) #data frame for individuals with highschool degree or GED
df_hs_ged = pd.read_sql(sql = hs_diploma, con = engine) #data frame for individuals with highschool degree or GED
df_no_degree = pd.read_sql(sql = no_degree, con = engine) #data frame for individuals with no degree
df_some_college = pd.read_sql(sql = some_college, con = engine) #data frame for individuals with some college experience
df_col_deg = pd.read_sql(sql = college_deg, con = engine) #data frame for individuals with an AA degree or higher

In [30]:
df_some_college['schl'].value_counts()

19    30
18    26
Name: schl, dtype: int64

In [18]:
df_total_oy['schl'].value_counts()

16    146
17     35
19     30
13     28
14     28
15     28
18     26
12     17
01     14
21     12
11     10
20      8
08      2
06      2
10      2
22      1
09      1
05      1
Name: schl, dtype: int64

In [31]:
df_hs_ged['schl'].value_counts()

16    146
17     35
Name: schl, dtype: int64

In [32]:
df_no_degree['schl'].value_counts()

14    28
13    28
15    28
12    17
01    14
11    10
10     2
06     2
08     2
09     1
05     1
Name: schl, dtype: int64

### functions used for the project

In [56]:
def get_average(data, col_name, age1 = 16, age2 = 24):
    try: 
        filtered_data = data[(data.agep >= age1) & (data.agep <= age2)]
        filtered_data['weighted_sum'] = filtered_data[col_name]*filtered_data['pwgtp']
        sum_of_weighted_data = filtered_data['weighted_sum'].sum()
        sum_of_weights = filtered_data['pwgtp'].sum()
        return sum_of_weighted_data/sum_of_weights
    except:
        print("operation could not be completed")


def weight_sum(df):
    weighted_sum = df['pwgtp'].sum()
    return weighted_sum
                      

def trisect_ages(df):
    df_16_18 = df[(df.agep>=16) & (df.agep <= 18)]
    value_16_18 = weight_sum(df_16_18)
    
    df_19_21 = df[(df.agep>=19) & (df.agep <= 21)]
    value_19_21 = weight_sum(df_19_21)
    
    df_22_24 = df[(df.agep>=22) & (df.agep <= 24)]
    value_22_24 = weight_sum(df_22_24)
    
    value_total = weight_sum(df)
    
    value_array = np.array([value_16_18, value_19_21, value_22_24, value_total])
    return value_array

def form_another_2d_array(df_list):
    new_list = np.zeros(len(df_list)+1)
    final_table = np.array([trisect_ages(x_df) for x_df in df_list])
    
#     for x in range(len(df_list)+1):
#         for y in range(len(df_list[0])+1)
#         element = df_table[0][x]-(df_table[1][x]+df_table[2][x]+df_table[3][x]+df_table[4][x])
#         print(element)
#         new_list[x] = element
#     final_table = np.append(df_table, [new_list], axis = 0)
    
    return final_table
    

def create_df(array_entry, columns_entry, index_entry):
    df = pd.DataFrame(array_entry, columns = columns_entry, index = index_entry)
    return df

def add_percentages(df):
    for col in df.columns:
        if 'total' in col:
            df["{} percentage".format(col[:5])] = round(df[col]/df.loc['Total Population', col]*100)
    

### creating the database

In [47]:
tri_sected1 = trisect_ages(df_col_deg)
tri_sected2 = trisect_ages(df_no_degree)
tri_sected5 = trisect_ages(df_total_oy)
tri_sected5


array([ 1815.,  3902.,  4897., 10614.])

In [49]:
second_array = form_another_2d_array([df_total_oy, df_no_degree, df_hs_ged, df_some_college, df_col_deg])
second_array

array([[ 1815.,  3902.,  4897., 10614.],
       [  916.,  1112.,  1349.,  3377.],
       [  781.,  2176.,  2135.,  5092.],
       [  118.,   521.,  1000.,  1639.],
       [    0.,    93.,   413.,   506.]])

In [44]:
column_names = ['16-18 total', '19-21 total', '22-24 total', '16-24 total']

In [58]:
index_names = ['Total Population', 'No HS Degree or GED', 'High School Degree/GED','Some College', 'AA or higher']

In [59]:
second_df = create_df(second_array, column_names, index_names)

In [60]:
second_df

,16-18 total,19-21 total,22-24 total,16-24 total
Total Population,1815.0,3902.0,4897.0,10614.0
No HS Degree or GED,916.0,1112.0,1349.0,3377.0
High School Degree/GED,781.0,2176.0,2135.0,5092.0
Some College,118.0,521.0,1000.0,1639.0
AA or higher,0.0,93.0,413.0,506.0


In [61]:
reorganized_list = ['16-18 percentage','16-18 total', '19-21 percentage', '19-21 total', '22-24 percentage', '22-24 total', '16-24 percentage', '16-24 total', ]

In [62]:
second_df = second_df.reindex(columns = reorganized_list)
second_df



,16-18 percentage,16-18 total,19-21 percentage,19-21 total,22-24 percentage,22-24 total,16-24 percentage,16-24 total
Total Population,NaN,1815.0,NaN,3902.0,NaN,4897.0,NaN,10614.0
No HS Degree or GED,NaN,916.0,NaN,1112.0,NaN,1349.0,NaN,3377.0
High School Degree/GED,NaN,781.0,NaN,2176.0,NaN,2135.0,NaN,5092.0
Some College,NaN,118.0,NaN,521.0,NaN,1000.0,NaN,1639.0
AA or higher,NaN,0.0,NaN,93.0,NaN,413.0,NaN,506.0


In [63]:
add_percentages(second_df)

In [64]:
second_df

,16-18 percentage,16-18 total,19-21 percentage,19-21 total,22-24 percentage,22-24 total,16-24 percentage,16-24 total
Total Population,100.0,1815.0,100.0,3902.0,100.0,4897.0,100.0,10614.0
No HS Degree or GED,50.0,916.0,28.0,1112.0,28.0,1349.0,32.0,3377.0
High School Degree/GED,43.0,781.0,56.0,2176.0,44.0,2135.0,48.0,5092.0
Some College,7.0,118.0,13.0,521.0,20.0,1000.0,15.0,1639.0
AA or higher,0.0,0.0,2.0,93.0,8.0,413.0,5.0,506.0
